In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13998,2024-12-24,Eua Nba,00:00,Denver Nuggets,Phoenix Suns,1.57,2.52,231.5,1.84,2.00,-4.5,1.93,1.93,niBq4qvP,0.636943,0.396825,0.543478,0.500000,0.033768,0.6,1.0,NaN,NaN,218.178,84.489810,0.387252,205.098,63.649866,0.310339,177.704,263.688,163.20,199.66,0.0,0.0,0.0,0.0,0.328485,0.058926,0.000000,-0.78,-0.156,-3.653846,0.685368,0.7,0.014632,-3.51,-0.702,-2.165242,0.518643,0.5,-0.018643
13999,2024-12-24,Eua Nba,00:00,Golden State Warriors,Indiana Pacers,1.47,2.82,230.5,1.84,2.00,-5.5,1.93,1.93,SpTi8ejt,0.680272,0.354610,0.543478,0.500000,0.034882,0.4,0.6,NaN,NaN,233.476,49.830150,0.213427,250.042,114.410129,0.457564,240.000,314.734,313.88,380.40,1.0,0.0,0.0,0.0,0.445032,0.058926,0.000000,-0.67,-0.134,-3.507463,0.557659,0.6,0.042341,2.76,0.552,3.297101,0.450336,0.4,-0.050336
14000,2024-12-24,Eua Nba,00:30,Los Angeles Lakers,Detroit Pistons,1.44,2.92,220.5,1.81,2.04,-5.5,1.93,1.93,zBWa6ZKh,0.694444,0.342466,0.552486,0.490196,0.036910,0.2,0.8,NaN,NaN,248.842,80.554536,0.323718,523.894,277.038260,0.528806,193.728,580.676,279.56,627.66,0.0,1.0,0.0,0.0,0.480054,0.084485,0.000000,-1.33,-0.266,-1.654135,0.668242,0.7,0.031758,2.86,0.572,3.356643,0.295193,0.3,0.004807
14001,2024-12-24,Austrália Nbl,03:30,Adelaide 36ers,Brisbane Bullets,1.67,2.22,194.5,1.81,1.85,-3.5,2.00,1.72,nkWqy9a6,0.598802,0.450450,0.552486,0.540541,0.049253,0.6,0.6,NaN,NaN,186.786,78.557222,0.420573,240.626,136.175900,0.565923,160.218,283.734,148.74,258.39,0.0,0.0,0.0,0.0,0.199953,0.015456,0.106446,0.59,0.118,5.677966,0.544414,0.8,0.255586,1.16,0.232,5.258621,0.383084,0.4,0.016916
14002,2024-12-24,Cazaquistão Liga Nacional,11:00,Aktobe,Tobol Kostanay,5.00,1.14,159.5,1.83,1.83,11.5,1.83,1.83,fFF9stYj,0.200000,0.877193,0.546448,0.546448,0.077193,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,350.00,90.48,1.0,0.0,0.0,0.0,0.889066,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
14003,2024-12-24,China Cba,08:35,Beijing Royal Fighters,Xinjiang,2.03,1.66,210.5,1.83,1.83,1.5,1.93,1.74,jiO2W31k,0.492611,0.602410,0.546448,0.546448,0.095020,0.0,0.0,NaN,NaN,206.208,110.976122,0.538176,138.010,29.422818,0.213193,166.668,155.144,117.72,136.35,0.0,0.0,0.0,0.0,0.141805,0.000000,0.073215,-0.02,-0.004,-257.500000,0.785633,0.8,0.014367,-1.07,-0.214,-3.084112,0.681215,0.5,-0.181215
14004,2024-12-24,China Cba,08:35,Shenzhen,Shanxi Zhongyu,3.03,1.31,217.5,1.83,1.83,5.5,2.01,1.68,OIyAUsW1,0.330033,0.763359,0.546448,0.546448,0.093392,0.2,1.0,NaN,NaN,388.406,150.576552,0.387678,210.212,100.988231,0.480411,315.464,215.444,165.41,326.70,0.0,0.0,0.0,0.0,0.560472,0.000000,0.126474,1.68,0.336,6.041667,0.521185,0.4,-0.121185,-0.01,-0.002,-155.000000,0.644890,0.6,-0.044890
14005,2024-12-24,China Wcba Feminina,08:30,Beijing F,Sichuan F,12.00,1.02,161.5,1.83,1.83,15.5,1.95,1.72,WGlgBBsH,0.083333,0.980392,0.546448,0.546448,0.063725,0.4,1.0,NaN,NaN,175.922,74.268823,0.422169,98.612,9.196191,0.093256,252.218,113.934,237.38,101.92,0.0,0.0,0.0,0.0,1.192632,0.000000,0.088629,-1.12,-0.224,-49.107143,0.605769,0.4,-0.205769,0.94,0.188,0.106383,0.864209,0.9,0.035791
14006,2024-12-24,China Wcba Feminina,08:30,Shaanxi Tianze F,Xiamen F,1.04,7.70,145.5,1.80,1.86,-17.5,2.04,1.66,M1eQ3TZj,0.961538,0.129870,0.555556,0.537634,0.091409,0.2,0.4,NaN,NaN,104.528,21.023994,0.201133,659.100,115.645552,0.175460,136.064,449.732,83.64,703.50,0.0,1.0,0.0,0.0,1.077650,0.023184,0.145244,2.61,0.522,0.076628,0.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Denver Nuggets,Phoenix Suns,231.5,1.84,0.9841,Over
1,00:00,Eua Nba,Golden State Warriors,Indiana Pacers,230.5,1.84,0.9842,Over
2,00:30,Eua Nba,Los Angeles Lakers,Detroit Pistons,220.5,1.81,0.9852,Over
